# <center><h1 style="font-family: Times New Roman; color: black;">Recommendation System</h1></center>

# Data Description:

Unique ID of each anime.

Anime title.

Anime broadcast type, such as TV, OVA, etc.

anime genre.

The number of episodes of each anime.

The average rating for each anime compared to the number of users who gave ratings.


# Number of community members for each anime.
Objective:
The objective of this assignment is to implement a recommendation system using cosine similarity on an anime dataset. 

Dataset:
Use the Anime Dataset which contains information about various anime, including their titles, genres,No.of episodes and user ratings etc.


# Tasks:

# 1. Data Preprocessing:

Load the dataset into a suitable data structure (e.g., pandas DataFrame).

Handle missing values, if any.

Explore the dataset to understand its structure and attributes.



# 2. Feature Extraction:

Decide on the features that will be used for computing similarity (e.g., genres, user ratings).

Convert categorical features into numerical representations if necessary.

Normalize numerical features if required.


# 3. Recommendation System:

Design a function to recommend anime based on cosine similarity.

Given a target anime, recommend a list of similar anime based on cosine similarity scores.

Experiment with different threshold values for similarity scores to adjust the recommendation list size.

# 4. Evaluation:

Split the dataset into training and testing sets.

Evaluate the recommendation system using appropriate metrics such as precision, recall, and F1-score.

Analyze the performance of the recommendation system and identify areas of improvement.

# Solutions: 

# 1. Data Preprocessing:

# 1.1 Load the dataset into a suitable data structure

In [1]:
# Import Libraries
import pandas as pd
import numpy as np

In [2]:
# Load Dataset
anime_data = pd.read_csv('anime.csv')

# Display the first few rows of the dataset
print("Initial Dataset:")
print(anime_data.head())

Initial Dataset:
   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   

   members  
0   200630  
1   793665  
2   114262  
3   673572  
4   151266  


# 1.2 Handle Missing Values

In [3]:
# Filling missing genres and types with 'Unknown' and dropping rows with missing ratings
anime_data['genre'].fillna('Unknown', inplace=True)
anime_data['type'].fillna('Unknown', inplace=True)
anime_data.dropna(subset=['rating'], inplace=True)

# Display missing values count after handling
print("\nMissing values after handling:")
print(anime_data.isnull().sum())


Missing values after handling:
anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64


# 1.3 Explore the Dataset Structure

In [4]:
# Summary statistics to understand the distribution of the data
print("\nDataset Summary:")
print(anime_data.describe(include='all'))


Dataset Summary:
            anime_id                     name   genre   type episodes  \
count   12064.000000                    12064   12064  12064    12064   
unique           NaN                    12062    3230      6      187   
top              NaN  Shi Wan Ge Leng Xiaohua  Hentai     TV        1   
freq             NaN                        2     816   3671     5612   
mean    13704.476044                      NaN     NaN    NaN      NaN   
std     11260.369521                      NaN     NaN    NaN      NaN   
min         1.000000                      NaN     NaN    NaN      NaN   
25%      3409.250000                      NaN     NaN    NaN      NaN   
50%     10004.000000                      NaN     NaN    NaN      NaN   
75%     23863.500000                      NaN     NaN    NaN      NaN   
max     34519.000000                      NaN     NaN    NaN      NaN   

              rating       members  
count   12064.000000  1.206400e+04  
unique           NaN           

# 2. Feature Extraction

# 2.1  Features that will be used for computing similarity

In [5]:
# Convert 'genre' into a format suitable for similarity calculations

# Splitting genre into multiple categories using One-Hot Encoding
genre_dummies = anime_data['genre'].str.get_dummies(sep=', ')

# Display the genre dummy columns
print("\nOne-Hot Encoded Genres:")
print(genre_dummies.head())


One-Hot Encoded Genres:
   Action  Adventure  Cars  Comedy  Dementia  Demons  Drama  Ecchi  Fantasy  \
0       0          0     0       0         0       0      1      0        0   
1       1          1     0       0         0       0      1      0        1   
2       1          0     0       1         0       0      0      0        0   
3       0          0     0       0         0       0      0      0        0   
4       1          0     0       1         0       0      0      0        0   

   Game  ...  Slice of Life  Space  Sports  Super Power  Supernatural  \
0     0  ...              0      0       0            0             1   
1     0  ...              0      0       0            0             0   
2     0  ...              0      0       0            0             0   
3     0  ...              0      0       0            0             0   
4     0  ...              0      0       0            0             0   

   Thriller  Unknown  Vampire  Yaoi  Yuri  
0         0      

# 2.2 Normalize 'Rating' and 'Members'

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Normalize the 'rating' and 'members' columns
anime_data[['rating', 'members']] = scaler.fit_transform(anime_data[['rating', 'members']])

# Combine genre dummies with the main dataset
anime_features = pd.concat([anime_data[['name', 'rating', 'members']], genre_dummies], axis=1)

# Display the first few rows of the processed dataset
print("\nProcessed Data for Similarity Calculation:")
print(anime_features.head())


Processed Data for Similarity Calculation:
                               name    rating   members  Action  Adventure  \
0                    Kimi no Na wa.  0.924370  0.197867       0          0   
1  Fullmetal Alchemist: Brotherhood  0.911164  0.782769       1          1   
2                          Gintama°  0.909964  0.112683       1          0   
3                       Steins;Gate  0.900360  0.664323       0          0   
4                     Gintama&#039;  0.899160  0.149180       1          0   

   Cars  Comedy  Dementia  Demons  Drama  ...  Slice of Life  Space  Sports  \
0     0       0         0       0      1  ...              0      0       0   
1     0       0         0       0      1  ...              0      0       0   
2     0       1         0       0      0  ...              0      0       0   
3     0       0         0       0      0  ...              0      0       0   
4     0       1         0       0      0  ...              0      0       0   

   Super Pow

# 3. Recommendation System

# 3.1 Design a Function to Recommend Anime Based on Cosine Similarity:

In [7]:
# Check if the anime name exists in the dataset
if 'Naruto' in anime_features['name'].values:
    print("'Naruto' is found in the dataset.")
else:
    print("'Naruto' is NOT found in the dataset.")

'Naruto' is found in the dataset.


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_anime(anime_name, data, top_n=5):
    # Compute cosine similarity between all anime
    cosine_sim = cosine_similarity(data.drop(columns=['name']))
    print(f"Cosine similarity matrix shape: {cosine_sim.shape}")
    
    # Get the index of the anime matching the name
    if anime_name not in data['name'].values:
        print(f"'{anime_name}' not found in the dataset.")
        return []
    
    idx = data.index[data['name'] == anime_name].tolist()[0]
    print(f"Index of '{anime_name}': {idx}")

    # Get similarity scores for the anime and sort by highest scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    print(f"Similarity scores: {sim_scores[:10]}")  # Print top 10 similarity scores

    # Get the indices of the top n similar anime
    top_anime_indices = [i[0] for i in sim_scores[1:top_n+1]]  # Exclude the target anime itself
    print(f"Top anime indices: {top_anime_indices}")

    # Return the names of the top n similar anime
    return data['name'].iloc[top_anime_indices]

# Test the function with debugging output
print(recommend_anime('Naruto', anime_features, top_n=5))

Cosine similarity matrix shape: (12064, 12064)
Index of 'Naruto': 841
Similarity scores: [(841, 1.0), (615, 0.9982223266342249), (1472, 0.9704748858769693), (1573, 0.9703560520127492), (486, 0.9694449560999671), (1343, 0.9639984785917441), (2997, 0.9637045880483297), (1103, 0.9632958954696472), (2458, 0.9625422050396012), (175, 0.8962613554319173)]
Top anime indices: [615, 1472, 1573, 486, 1343]
615                                    Naruto: Shippuuden
1472          Naruto: Shippuuden Movie 4 - The Lost Tower
1573    Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsu...
486                              Boruto: Naruto the Movie
1343                                          Naruto x UT
Name: name, dtype: object


# 3.2 Recommend a List of Similar Anime Based on Cosine Similarity Scores:

In [9]:
# Test the recommendation system by providing an anime title
print("\nTop 5 similar anime to 'Naruto':")
print(recommend_anime('Naruto', anime_features, top_n=5))


Top 5 similar anime to 'Naruto':
Cosine similarity matrix shape: (12064, 12064)
Index of 'Naruto': 841
Similarity scores: [(841, 1.0), (615, 0.9982223266342249), (1472, 0.9704748858769693), (1573, 0.9703560520127492), (486, 0.9694449560999671), (1343, 0.9639984785917441), (2997, 0.9637045880483297), (1103, 0.9632958954696472), (2458, 0.9625422050396012), (175, 0.8962613554319173)]
Top anime indices: [615, 1472, 1573, 486, 1343]
615                                    Naruto: Shippuuden
1472          Naruto: Shippuuden Movie 4 - The Lost Tower
1573    Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsu...
486                              Boruto: Naruto the Movie
1343                                          Naruto x UT
Name: name, dtype: object


# 3.3 Experiment with Different Threshold Values for Similarity Scores:

In [10]:
def recommend_anime(anime_name, data, top_n=5, threshold=0.0):
    # Compute cosine similarity between all anime
    cosine_sim = cosine_similarity(data.drop(columns=['name']))
    
    # Get the index of the anime matching the name
    idx = data.index[data['name'] == anime_name].tolist()[0]

    # Get similarity scores for the anime and sort by highest scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Filter recommendations by threshold and get top_n similar anime
    sim_scores = [i for i in sim_scores[1:] if i[1] >= threshold]  # Exclude the target anime itself
    top_anime_indices = [i[0] for i in sim_scores[:top_n]]  # Take top_n after threshold filtering

    # Return the names of the top similar anime
    return data['name'].iloc[top_anime_indices]

# Example with threshold
print("\nTop 5 similar anime to 'Naruto' with a similarity threshold of 0.5:")
print(recommend_anime('Naruto', anime_features, top_n=5, threshold=0.5))


Top 5 similar anime to 'Naruto' with a similarity threshold of 0.5:
615                                    Naruto: Shippuuden
1472          Naruto: Shippuuden Movie 4 - The Lost Tower
1573    Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsu...
486                              Boruto: Naruto the Movie
1343                                          Naruto x UT
Name: name, dtype: object


# 4. Evaluation:

# 4.1. Split the Dataset into Training and Testing Sets

In [11]:
from sklearn.model_selection import train_test_split

# 1.1 Split the dataset into training and testing sets
train_data, test_data = train_test_split(anime_features, test_size=0.2, random_state=42)

# Display the size of the training and testing sets
print(f"Training set size: {len(train_data)}")
print(f"Testing set size: {len(test_data)}")

Training set size: 9651
Testing set size: 2413


# 4.2. Evaluate the Recommendation System

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_anime(anime_name, data, top_n=5):
    # Compute cosine similarity between all anime
    cosine_sim = cosine_similarity(data.drop(columns=['name']))
    print(f"Cosine similarity matrix shape: {cosine_sim.shape}")
    
    # Check if the anime name exists (allow case-insensitive matching)
    matching_anime = data[data['name'].str.contains(anime_name, case=False, regex=False)]
    if matching_anime.empty:
        print(f"'{anime_name}' not found in the dataset.")
        return []
    
    # Get the index of the anime matching the name
    idx = matching_anime.index[0]
    print(f"Index of '{anime_name}': {idx}")

    # Get similarity scores for the anime and sort by highest scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    print(f"Similarity scores for '{anime_name}': {sim_scores[:10]}")  # Print top 10 similarity scores

    # Get the indices of the top n similar anime
    top_anime_indices = [i[0] for i in sim_scores[1:top_n+1]]  # Exclude the target anime itself
    print(f"Top anime indices: {top_anime_indices}")

    # Return the names of the top n similar anime
    return data['name'].iloc[top_anime_indices]

# Test the function with a specific anime
recommended_anime = recommend_anime('Naruto', anime_features, top_n=5)
print("\nTop 5 similar anime to 'Naruto':")
print(recommended_anime)

Cosine similarity matrix shape: (12064, 12064)
Index of 'Naruto': 486
Similarity scores for 'Naruto': [(486, 1.0), (1472, 0.9996970721209669), (1573, 0.9996622124969153), (1103, 0.9995668019955601), (1343, 0.9995357644996914), (2458, 0.998968660433799), (2997, 0.9987831703466037), (615, 0.9823159410035079), (841, 0.9694449560999671), (175, 0.9028506192791678)]
Top anime indices: [1472, 1573, 1103, 1343, 2458]

Top 5 similar anime to 'Naruto':
1472          Naruto: Shippuuden Movie 4 - The Lost Tower
1573    Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsu...
1103    Boruto: Naruto the Movie - Naruto ga Hokage ni...
1343                                          Naruto x UT
2458                 Naruto Shippuuden: Sunny Side Battle
Name: name, dtype: object


<center>
    <h1 style="font-family: 'Cursive'; 
               font-size: 150px; 
               color: Yellow; 
               text-shadow: 0 0 10px #0000FF, 0 0 20px #0000FF, 0 0 30px #0000FF;">
        Thank You
    </h1>
</center>